In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn import svm
import pandas as pd
from IPython.display import display

from __future__ import print_function
import os
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.utils as vutils
import math

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
def setup_data(dir, num_circuits, file_prefix, dir_with_gates, separate_bits, norm = 'max_norm'):
  onlyfiles = next(os.walk(dir))[2] #dir is your directory path as string

  # Currently, we only count files with frequencies
  if dir_with_gates:
    num_files = int(math.ceil((len(onlyfiles) - 1)/3))
  else:
    num_files = len(onlyfiles)

  for i in range(num_files): # Only FC files matter and there is an extra IDs file
    # Give unique indexes to each circuit in each file
    idx = i*num_circuits
    d = np.load(dir + "/" + file_prefix + str(i) + ".npy", allow_pickle=True)
    circuits = np.concatenate(d[:,0]).reshape(d.shape[0],2) + np.array([idx,idx+ int(num_circuits/2)]) 

    # Separate out cicruits and freq
    d = d[:,1:]

    # Stack
    if i == 0:
      det_c = np.copy(d)
      circ = np.copy(circuits)
    else:
      det_c = np.vstack((det_c, d))
      circ = np.vstack((circ, circuits))

  # Separate out frequencies by setting on each bit (instead of settings on pairs)
  if separate_bits == "True":
    det_c_copy = np.empty(det_c.shape)

    # 0 on qubit_0
    det_c_copy[:,0] = det_c[:,0] + det_c[:,2]

    # 1 on qubit_0
    det_c_copy[:,1] = det_c[:,1] + det_c[:,3]

    # 0 on qubit_1
    det_c_copy[:,2] = det_c[:,0] + det_c[:,1]

    # 1 on qubit_1
    det_c_copy[:,3] = det_c[:,2] + det_c[:,3]

    det_c = det_c_copy



  # Normalize frequencies
  if norm == "max_norm":
    det_c = det_c/np.max(det_c)
  elif norm == "col_max_norm":
    det_c = det_c/det_c.max(axis = 0)
  elif norm == "row_norm":
    det_c = det_c/np.sum(det_c[0,:])

  # Used only for testing
  elif norm == "no_norm":
    det_c = det_c

  return circ, det_c

def create_labels(label, size):
  return np.full((size,), label)

def create_dataset(features, label):
  y = create_labels(label, features.shape[0])
  return features, y

def combine_datasets(x1, y1, x2, y2):
  return np.vstack((x1,x2)), np.concatenate((y1,y2))

def binary_comparison_data_pipeline(freq_1, freq_2, test_frac = 0.2):
  x1, y1 = create_dataset(freq_1, 1)
  x2, y2 = create_dataset(freq_2, 0)
  x, y = combine_datasets(x1,y1, x2, y2)
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

def svm_expt(freq_1, freq_2, test_size = 0.2):
  X_train, X_test, y_train, y_test = binary_comparison_data_pipeline(freq_1, freq_2, test_frac = test_size)

  clf = svm.SVC()
  clf.fit(X_train, y_train)

  test_acc = (clf.predict(X_test) == y_test).mean()

  return test_acc

# Apply binary expt to all possible combinations of data_list and pretty print
def create_expt_matrix(dataset_list, data_list, binary_expt):
  num_datasets = len(data_list)

  # get names of datasets
  dataset_names = [d['dir_name'] for d in dataset_list]

  expt_matrix = np.zeros((num_datasets, num_datasets))
  for i in range(num_datasets):
    for j in range(num_datasets):
      if (j >= i) :
        expt_matrix[i][j] = binary_expt(data_list[i][1], data_list[j][1])

  # pretty printing
  df = pd.DataFrame(expt_matrix, index=dataset_names, columns=dataset_names)
  display(df)

  return expt_matrix

# Setup a list of data to be experimented on

# dir_prefix - path to directory where all data is stored
# f_prefix - prefix of file names with frequency data
# dir_with_gates - True if data directory has gates included
# sep_bits - if True, setup data such that frequency is calculated on each result bit instead of result pairs
# normalize - Normalization scheme used
def setup_datalist(datasets, dir_prefix, f_prefix, sep_bits, normalize):
  data_list = []
  data_list.append(setup_data(dir_prefix + datasets[0]['dir_name'],  num_circuits = datasets[0]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[0]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[1]['dir_name'],  num_circuits = datasets[1]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[1]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[2]['dir_name'],  num_circuits = datasets[2]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[2]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[3]['dir_name'],  num_circuits = datasets[3]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[3]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[4]['dir_name'],  num_circuits = datasets[4]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[4]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[5]['dir_name'],  num_circuits = datasets[5]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[5]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[6]['dir_name'],  num_circuits = datasets[6]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[6]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[7]['dir_name'],  num_circuits = datasets[7]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[7]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[8]['dir_name'],  num_circuits = datasets[8]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[8]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))

  return data_list

In [ ]:
ideal_circuits, ideal_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates", norm = "row_norm")

In [ ]:
ideal_freq.sum(axis = 1)

array([1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0], dtype=object)

In [ ]:
ideal_circuits, ideal_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates", separate_bits = "True", norm = "row_norm")

In [ ]:
ideal_freq.sum(axis = 1)

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
xt2WithGates_circuits, xt2WithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates")

100


KeyboardInterrupt: ignored

## Normalization Schemes


1.   max_norm - takes the maximum over the entire frequency matrix (across all circuit settings and all result bit settings)
2.   col_max_norm - takes the columnwise maximum (i.e max across all circuit settings for a particular result setting)
3. row_norm - uses the row-wise sum (i.e total number of times a circuit setting was simulated)



## Metadata of Datasets used

In [4]:
datasets = [
            {
              'dir_name':"IdealGates",
              'num_circuits':20,
              'dir_with_gates':True
            },
            {
              'dir_name':"Operation Crosstalk 1 (a)",
              'num_circuits':20,
              'dir_with_gates':False
            },
            {
              'dir_name':"Operation Crosstalk 2 (b)",
              'num_circuits':20,
              'dir_with_gates':False
            },
            {
              'dir_name':"Detection Crosstalk (c)",
              'num_circuits':40,
              'dir_with_gates':False
            },
            {
              'dir_name':"XT1WithGates",
              'num_circuits':20,
              'dir_with_gates':True
            },
            {
              'dir_name':"XT2WithGates",
              'num_circuits':20,
              'dir_with_gates':True
            },
            {
              'dir_name':"DetX400",
              'num_circuits':40,
              'dir_with_gates':True
            },
            {
              'dir_name':"DetXWithGates",
              'num_circuits':40,
              'dir_with_gates':True
            },
            {
              'dir_name':"DetX100",
              'num_circuits':20,
              'dir_with_gates':True
            }
]

## Expts with bit frequencies not separated

In [ ]:
ideal_circuits, ideal_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates")
a_circuits, a_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 1 (a)", dir_with_gates = False)
b_circuits, b_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 2 (b)", dir_with_gates = False)
c_circuits, c_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Detection Crosstalk (c)",num_circuits=40, dir_with_gates = False)
detx400_circuits, detx400_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX400", num_circuits=40)
detxWithGates_circuits, detxWithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetXWithGates", num_circuits=40)
detx100_circuits, detx100_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX100")
xt1WithGates_circuits, xt1WithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates")
xt2WithGates_circuits, xt2WithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT2WithGates")

In [5]:
dir_prefix = "gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/"
f_prefix = "FCTexts_"


### max_norm

In [ ]:
separate_bits = False
norm = "max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [ ]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.99325,0.51575,1.000000,0.99875,0.48800,0.795437,1.000000,0.504750
Operation Crosstalk 1 (a),0.000,0.47950,0.99050,1.000000,1.00000,0.99075,0.992460,1.000000,0.988250
Operation Crosstalk 2 (b),0.000,0.00000,0.48550,1.000000,0.99850,0.50075,0.795437,1.000000,0.500250
Detection Crosstalk (c),0.000,0.00000,0.00000,0.487875,1.00000,1.00000,1.000000,0.518346,1.000000
XT1WithGates,0.000,0.00000,0.00000,0.000000,0.48800,0.99875,0.999702,1.000000,0.999000
XT2WithGates,0.000,0.00000,0.00000,0.000000,0.00000,0.48800,0.795437,1.000000,0.515750
DetX400,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.492450,1.000000,0.804067
DetXWithGates,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,1.000000
DetX100,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


### col_max_norm

In [ ]:
separate_bits = False
norm = "col_max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [ ]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.7900,0.51575,1.000000,0.99875,0.48800,0.795437,1.000000,0.548000
Operation Crosstalk 1 (a),0.000,0.4795,0.78525,1.000000,0.91775,0.77925,0.869643,1.000000,0.808250
Operation Crosstalk 2 (b),0.000,0.0000,0.48550,1.000000,0.99875,0.50075,0.795437,1.000000,0.516250
Detection Crosstalk (c),0.000,0.0000,0.00000,0.487875,1.00000,1.00000,1.000000,0.518346,1.000000
XT1WithGates,0.000,0.0000,0.00000,0.000000,0.48800,0.99900,0.999603,1.000000,0.999000
XT2WithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.48800,0.795437,1.000000,0.570000
DetX400,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.492450,1.000000,0.804067
DetXWithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,1.000000
DetX100,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


### row_norm

In [ ]:
separate_bits = False
norm = "row_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [ ]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.6940,0.51575,0.910000,0.65250,0.48800,0.795437,0.909623,0.504750
Operation Crosstalk 1 (a),0.000,0.4795,0.68925,0.917800,0.55375,0.68825,0.868552,0.918155,0.690750
Operation Crosstalk 2 (b),0.000,0.0000,0.48550,0.910600,0.66025,0.50075,0.795437,0.910714,0.499500
Detection Crosstalk (c),0.000,0.0000,0.00000,0.487875,0.91250,0.91250,0.790112,0.518346,0.911000
XT1WithGates,0.000,0.0000,0.00000,0.000000,0.48800,0.65275,0.861111,0.909623,0.707750
XT2WithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.48800,0.795437,0.909623,0.509750
DetX400,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.492450,0.779889,0.804067
DetXWithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,0.915079
DetX100,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


## Expts with bit frequencies separated

### max_norm

In [ ]:
separate_bits = True
norm = "max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [ ]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.99325,0.51575,1.000000,0.99875,0.48800,0.795437,1.000000,0.504750
Operation Crosstalk 1 (a),0.000,0.47950,0.99050,1.000000,1.00000,0.99075,0.992460,1.000000,0.988250
Operation Crosstalk 2 (b),0.000,0.00000,0.48550,1.000000,0.99850,0.50075,0.795437,1.000000,0.500250
Detection Crosstalk (c),0.000,0.00000,0.00000,0.487875,1.00000,1.00000,1.000000,0.518346,1.000000
XT1WithGates,0.000,0.00000,0.00000,0.000000,0.48800,0.99875,0.999702,1.000000,0.999000
XT2WithGates,0.000,0.00000,0.00000,0.000000,0.00000,0.48800,0.795437,1.000000,0.515750
DetX400,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.492450,1.000000,0.804067
DetXWithGates,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,1.000000
DetX100,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


### col_max_norm

In [ ]:
separate_bits = True
norm = "col_max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [ ]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.7900,0.51575,1.000000,0.99875,0.48800,0.795437,1.000000,0.548000
Operation Crosstalk 1 (a),0.000,0.4795,0.78525,1.000000,0.91775,0.77925,0.869643,1.000000,0.808250
Operation Crosstalk 2 (b),0.000,0.0000,0.48550,1.000000,0.99875,0.50075,0.795437,1.000000,0.516250
Detection Crosstalk (c),0.000,0.0000,0.00000,0.487875,1.00000,1.00000,1.000000,0.518346,1.000000
XT1WithGates,0.000,0.0000,0.00000,0.000000,0.48800,0.99900,0.999603,1.000000,0.999000
XT2WithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.48800,0.795437,1.000000,0.570000
DetX400,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.492450,1.000000,0.804067
DetXWithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,1.000000
DetX100,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


### row_norm

In [ ]:
separate_bits = True
norm = "row_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [ ]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.6940,0.51575,0.910000,0.65250,0.48800,0.795437,0.909623,0.504750
Operation Crosstalk 1 (a),0.000,0.4795,0.68925,0.917800,0.55375,0.68825,0.868552,0.918155,0.690750
Operation Crosstalk 2 (b),0.000,0.0000,0.48550,0.910600,0.66025,0.50075,0.795437,0.910714,0.499500
Detection Crosstalk (c),0.000,0.0000,0.00000,0.487875,0.91250,0.91250,0.790112,0.518346,0.911000
XT1WithGates,0.000,0.0000,0.00000,0.000000,0.48800,0.65275,0.861111,0.909623,0.707750
XT2WithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.48800,0.795437,0.909623,0.509750
DetX400,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.492450,0.779889,0.804067
DetXWithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,0.915079
DetX100,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


## Transfer Learning

In [ ]:
def transfer_svm_expt(freq_1, freq_2, freq_3):
  # freq_1 gets label 1, freq_2 gets label 0
  # freq_3 is assumed to have label 1 (this is the dataset we are transferrig to)
  # No need for a test set since we are performing transfer learning
  X_train, X_test, y_train, y_test = binary_comparison_data_pipeline(freq_1, freq_2, test_frac = 0.0)

  clf = svm.SVC()
  clf.fit(X_train, y_train)

  X_transfer, y_transfer = create_dataset(freq_3, 1)

  transfer_acc = (clf.predict(X_transfer) == y_transfer).mean()

  return transfer_acc

In [ ]:
# Apply binary expt to all possible combinations of data_list and pretty print

# Rows - trained on
# Columns - transferred to

def create_transfer_expt_matrix(dataset_list, data_list, binary_expt):
  num_datasets = len(data_list) 

  # get names of datasets
  dataset_names = [d['dir_name'] for d in dataset_list]

  expt_matrix = np.zeros((num_datasets - 1, num_datasets - 1))

  for i in range(1, num_datasets):
    for j in range(1, num_datasets):
      expt_matrix[i-1][j-1] = binary_expt(data_list[i][1], data_list[0][1], data_list[j][1])

  # pretty printing
  df = pd.DataFrame(expt_matrix, index=dataset_names[1:], columns=dataset_names[1:])
  display(df)

  return expt_matrix

In [ ]:
separate_bits = True
norm = "row_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [ ]:
svm_expt_matrix = create_transfer_expt_matrix(datasets, data_list, transfer_svm_expt)

,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
Operation Crosstalk 1 (a),0.4737,0.1069,0.111650,0.4236,0.1133,0.112203,0.112302,0.1110
Operation Crosstalk 2 (b),0.4256,0.3977,0.395625,0.3689,0.3689,0.386584,0.377277,0.3738
Detection Crosstalk (c),0.4003,0.4437,1.000000,0.4616,0.4616,0.442252,1.000000,0.4632
XT1WithGates,0.3482,0.0000,0.000000,0.3097,0.0000,0.000000,0.000000,0.0000
XT2WithGates,0.3455,0.3036,0.310325,0.2847,0.2847,0.297005,0.306559,0.2924
DetX400,1.0000,1.0000,0.723325,1.0000,1.0000,1.000000,0.739876,1.0000
DetXWithGates,0.4003,0.4437,1.000000,0.4616,0.4616,0.442252,1.000000,0.4632
DetX100,0.6869,0.6957,0.661375,0.6936,0.6922,0.679604,0.685149,0.7051


In [ ]:
separate_bits = True
norm = "max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [ ]:
svm_expt_matrix = create_transfer_expt_matrix(datasets, data_list, transfer_svm_expt)

,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
Operation Crosstalk 1 (a),0.9798,0.0000,0.884550,0.0764,0.0000,0.000000,0.893391,0.0000
Operation Crosstalk 2 (b),0.4256,0.3977,0.395625,0.3689,0.3689,0.386584,0.377277,0.3738
Detection Crosstalk (c),0.0000,0.0000,1.000000,0.0000,0.0000,0.000000,1.000000,0.0000
XT1WithGates,0.2871,0.0000,0.000000,0.9981,0.0000,0.000000,0.000000,0.0000
XT2WithGates,0.3455,0.3036,0.310325,0.2847,0.2847,0.297005,0.306559,0.2924
DetX400,1.0000,1.0000,0.946500,0.9851,1.0000,1.000000,0.948119,1.0000
DetXWithGates,0.0000,0.0000,1.000000,0.0000,0.0000,0.000000,1.000000,0.0000
DetX100,0.6994,0.6957,0.692575,0.6921,0.6922,0.679604,0.715149,0.7051


In [ ]:
separate_bits = True
norm = "col_max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [ ]:
svm_expt_matrix = create_transfer_expt_matrix(datasets, data_list, transfer_svm_expt)

,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
Operation Crosstalk 1 (a),0.6659,0.1188,0.111650,0.7027,0.1270,0.112228,0.112302,0.1111
Operation Crosstalk 2 (b),0.4256,0.3977,0.395625,0.3689,0.3689,0.386584,0.377277,0.3738
Detection Crosstalk (c),0.0000,0.0000,1.000000,0.0000,0.0000,0.000000,1.000000,0.0000
XT1WithGates,0.4347,0.0000,0.000000,0.9984,0.0000,0.000000,0.000000,0.0000
XT2WithGates,0.3455,0.3036,0.188850,0.2847,0.2847,0.297005,0.189975,0.2924
DetX400,1.0000,1.0000,0.061950,1.0000,1.0000,1.000000,0.054728,1.0000
DetXWithGates,0.0000,0.0000,1.000000,0.0000,0.0000,0.000000,1.000000,0.0000
DetX100,0.4622,0.4952,0.031200,0.6282,0.5114,0.679431,0.030000,0.7051


## MADE

In [3]:
hidden_sizes = [512, 512, 512]
nin = 784

L = len(hidden_sizes)

m = {}

# sample the order of the inputs and the connectivity of all neurons
m[-1] = np.arange(nin)
for l in range(L):
  # min so that we don't have indep node in between
  m[l] = np.random.randint(m[l - 1].min(), nin - 1, size=hidden_sizes[l])

# construct the mask matrices
masks = [m[l - 1][:, None] <= m[l][None, :] for l in range(L)]

# No equality bcoz we cant have variable dependent on itself
masks.append(m[L - 1][:, None] < m[-1][None, :])


In [4]:
m[-1][:,None].shape

(784, 1)

In [15]:
(m[-1][:,None] <= m[0][None,:]).shape

(784, 512)

In [13]:
m[0][None,:].shape

(1, 512)

In [2]:
np.repeat(np.array([0,1]), 25, axis = 1)

AxisError: ignored

In [46]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def to_one_hot(labels, d):
  one_hot = torch.FloatTensor(labels.shape[0], d).cuda()
  one_hot.zero_()
  one_hot.scatter_(1, labels.unsqueeze(1), 1)
  return one_hot

# Code based one Andrej Karpathy's implementation: https://github.com/karpathy/pytorch-made
class MaskedLinear(nn.Linear):
  def __init__(self, in_features, out_features, bias=True):
    super().__init__(in_features, out_features, bias)
    self.register_buffer('mask', torch.ones(out_features, in_features))

  def set_mask(self, mask):
    self.mask.data.copy_(torch.from_numpy(mask.astype(np.uint8).T))

  def forward(self, input):
    return F.linear(input, self.mask * self.weight, self.bias)

class MADE(nn.Module):
  def __init__(self, input_shape, d, hidden_size=[512, 512, 512], 
               ordering=None, one_hot_input=False):
    super().__init__()
    self.input_shape = input_shape
    self.nin = np.prod(input_shape)
    self.nout = self.nin * d
    self.d = d
    self.hidden_sizes = hidden_size
    self.ordering = np.arange(self.nin) if ordering is None else ordering
    self.one_hot_input = one_hot_input

    # define a simple MLP neural net
    self.net = []
    hs = [self.nin * d if one_hot_input else self.nin] + self.hidden_sizes + [self.nout]
    for h0, h1 in zip(hs, hs[1:]):
      self.net.extend([
        MaskedLinear(h0, h1),
        nn.ReLU(),
      ])
    self.net.pop()  # pop the last ReLU for the output layer
    self.net = nn.Sequential(*self.net)

    self.m = {}
    self.create_mask()  # builds the initial self.m connectivity

  def create_mask(self):
    L = len(self.hidden_sizes)

    # sample the order of the inputs and the connectivity of all neurons
    self.m[-1] = np.array([0,1])
    for l in range(L):
      self.m[l] = np.random.randint(self.m[l - 1].min(), 
                                      self.nin/2 - 1, size=int(self.hidden_sizes[l]/2))

    # construct the mask matrices
    masks = [self.m[l - 1][:, None] <= self.m[l][None, :] for l in range(L)]
    masks.append(self.m[L - 1][:, None] < self.m[-1][None, :])

    masks[-1] = np.repeat(masks[-1], self.d, axis=1)
    if self.one_hot_input:
      masks[0] = np.repeat(masks[0], self.d, axis=0)

    final_masks = []
    for i in range(len(masks)):
      mask = masks[i]
      zero_arr = np.zeros(mask.shape)
      prefix = np.hstack((zero_arr, mask))
      suffix = np.hstack((mask, zero_arr))
      final_masks.append(np.vstack((suffix, prefix)))

    # set the masks in all MaskedLinear layers
    layers = [l for l in self.net.modules() if isinstance(l, MaskedLinear)]
    for l, m in zip(layers, final_masks):
      l.set_mask(m)

  def forward(self, x):
    batch_size = x.shape[0]
    if self.one_hot_input:
      x = x.long().contiguous().view(-1)
      x = to_one_hot(x, self.d)
      x = x.view(batch_size, -1)
    else:
      x = x.float()
      x = x.view(batch_size, self.nin)
    logits = self.net(x).view(batch_size, self.nin, self.d)
    return logits.permute(0, 2, 1).contiguous().view(batch_size, self.d, *self.input_shape)

  def loss(self, x):
      return F.cross_entropy(self(x), x.long())

  def sample(self, n):
    samples = torch.zeros(n, self.nin).cuda()
    self.inv_ordering = {x: i for i, x in enumerate(self.ordering)}
    with torch.no_grad():
      for i in range(self.nin):
        logits = self(samples).view(n, self.d, self.nin)[:, :, self.inv_ordering[i]]
        probs = F.softmax(logits, dim=1)
        samples[:, self.inv_ordering[i]] = torch.multinomial(probs, 1).squeeze(-1)
      samples = samples.view(n, *self.input_shape)
    return samples.cpu().numpy()

In [8]:
def gen_data_2(data, circ, frac):
  data = data * (frac)
  data = data.astype('int')
  ds_size =  np.sum(data)
  ds_size_column = np.cumsum(data, axis = 1)
  arr = np.zeros((ds_size, 4))

  running_sum = 0
  for i in range(circ.shape[0]):
    arr[running_sum :running_sum + ds_size_column[i][0],:] = [circ[i][0],0,circ[i][1],0]
    arr[running_sum + ds_size_column[i][0]:running_sum + ds_size_column[i][1],:] = [circ[i][0],1,circ[i][1],0]
    arr[running_sum + ds_size_column[i][1]:running_sum + ds_size_column[i][2],:] = [circ[i][0],0,circ[i][1],1]
    arr[running_sum + ds_size_column[i][2]:running_sum + ds_size_column[i][3],:] = [circ[i][0],1,circ[i][1],1]
    running_sum += np.sum(data[i])
  np.random.shuffle(arr)
  return arr

In [37]:
circ, freq_data = setup_data(dir_prefix + datasets[0]['dir_name'],  num_circuits = datasets[0]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[0]['dir_with_gates'], separate_bits = False, norm = 'no_norm')

In [38]:
a = gen_data_2(freq_data[:100], circ[:100], 0.001)
a.shape

(9795, 4)

In [55]:
def gen_data(data, circ, frac):
  data = data * (frac)
  data = data.astype('int')
  ds_size =  np.sum(data)
  ds_size_column = np.cumsum(np.sum(data, axis = 0))
  print(np.sum(data, axis = 0))
  arr = np.zeros((ds_size, 4))
  arr[:ds_size_column[0],:] = [0,0,0,0]
  arr[ds_size_column[0]:ds_size_column[1],:] = [0,1,0,0]
  arr[ds_size_column[1]:ds_size_column[2],:] = [0,0,0,1]
  arr[ds_size_column[2]:ds_size_column[3],:] = [0,1,0,1]
  np.random.shuffle(arr)
  return arr

def train(model, train_loader, optimizer, epoch, grad_clip=None):
  model.train()
  
  train_losses = []
  for x in train_loader:
    x = x.cuda().contiguous()
    loss = model.loss(x)
    optimizer.zero_grad()
    loss.backward()
    if grad_clip:
      torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    optimizer.step()
    train_losses.append(loss.item())
  return train_losses

def eval_loss(model, data_loader):
  model.eval()
  total_loss = 0
  with torch.no_grad():
    for x in data_loader:
      x = x.cuda().contiguous()
      loss = model.loss(x)
      total_loss += loss * x.shape[0]
    avg_loss = total_loss / len(data_loader.dataset)

  return avg_loss.item()

def train_epochs(model, train_loader, test_loader, train_args, quiet = True):
  epochs, lr = train_args['epochs'], train_args['lr']
  grad_clip = train_args.get('grad_clip', None)
  optimizer = optim.Adam(model.parameters(), lr=lr)

  train_losses = []
  test_losses = [eval_loss(model, test_loader)]
  for epoch in range(epochs):
    model.train()
    train_loss = train(model, train_loader, optimizer, epoch, grad_clip)
    train_losses.extend(train_loss)
    test_loss = eval_loss(model, test_loader)
    test_losses.append(test_loss)
    if not quiet:
      print(f'Epoch {epoch}  Train Loss : {train_loss[0]}, {train_loss[-1]}  Test loss {test_loss:.4f}')

  return train_losses, test_losses

# frac - fraction of shots to pick, ex 0.001 would have 100 shots per pair of circuits
def MADE_expt(model, dataset, frac, data_fn, ep):

  # Load Dataset
  circ, freq_data = dataset[0], dataset[1]

  d = data_fn(freq_data, circ, frac)
  split = int(0.8 * len(d))
  train_data, test_data = d[:split], d[split:]

  train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=128)
  train_losses, test_losses = train_epochs(model, train_loader, test_loader, 
                                            dict(epochs=ep, lr=2e-3))
  return train_losses[-1], test_losses[-1]

In [67]:
MADE_expt(0.001, 0, dir_prefix, f_prefix)

[[43341 43586 6459 6614]
 [25076 25005 24917 25002]
 [24841 25072 24936 25151]
 [24949 24959 25065 25027]
 [25097 25032 24961 24910]
 [24962 25038 24965 25035]
 [24894 25121 24843 25142]
 [25047 25044 24936 24973]
 [24886 24875 25134 25105]
 [6564 6620 43425 43391]]
[240750 482597 731312 980050]
(784040, 4)
Epoch 0  Train Loss : 0.7642644643783569, 0.34820717573165894  Test loss 0.3477
Epoch 1  Train Loss : 0.34861886501312256, 0.3463367819786072  Test loss 0.3466
Epoch 2  Train Loss : 0.3468921482563019, 0.34552472829818726  Test loss 0.3466


In [68]:
MADE_expt(0.01, 4, dir_prefix, f_prefix)

[[4218 4205 783 794]
 [2499 2475 2562 2464]
 [2568 2431 2512 2489]
 [2479 2536 2492 2493]
 [2515 2502 2490 2493]
 [2496 2514 2502 2488]
 [2582 2512 2445 2461]
 [2505 2578 2483 2434]
 [2457 2451 2574 2518]
 [798 824 4158 4220]]
[241255 483420 731769 980295]
(784236, 4)
Epoch 0  Train Loss : 0.6942412257194519, 0.3478628695011139  Test loss 0.3475
Epoch 1  Train Loss : 0.3468514382839203, 0.3468070924282074  Test loss 0.3466
Epoch 2  Train Loss : 0.346502423286438, 0.34624841809272766  Test loss 0.3466


In [70]:
MADE_expt(0.001, 5, dir_prefix, f_prefix)

[[43704 43378 6339 6579]
 [25153 24941 25557 24349]
 [25165 24788 24907 25140]
 [25153 24992 25299 24556]
 [25025 24855 25040 25080]
 [24914 24960 25349 24777]
 [25014 25177 24854 24955]
 [25228 24450 24931 25391]
 [24927 25280 24892 24901]
 [6542 6454 43102 43902]]
[240743 482410 731134 979905]
(783924, 4)
Epoch 0  Train Loss : 0.6436841487884521, 0.34794971346855164  Test loss 0.3474
Epoch 1  Train Loss : 0.3475525677204132, 0.34641405940055847  Test loss 0.3466
Epoch 2  Train Loss : 0.3466276228427887, 0.3462686240673065  Test loss 0.3466


In [ ]:
dataset_idx = 0
ep = 15
num_lines = 100
num_circuits = 20

circ, freq = setup_data(dir_prefix + datasets[dataset_idx]['dir_name'],  num_circuits = datasets[dataset_idx]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[dataset_idx]['dir_with_gates'], separate_bits = False, norm = 'no_norm')

train_l, test_l = [], []
for i in range(100):
  c, f = circ[i*num_lines: (i+1)*num_lines], freq[i*num_lines: (i+1)*num_lines]
  c = c - i*num_circuits
  
  model = MADE((4,), 20, hidden_size=[160, 160], one_hot_input=True).cuda()
  final_train_loss, final_test_loss = MADE_expt(model, dataset, 0.01, gen_data_2, ep)
  train_l.append(final_train_loss)
  test_l.append(final_test_loss)

In [50]:
final_train_loss

1.4423638582229614

In [ ]:
dataset_idx = 4
ep = 10
data_idx = 100

circ, freq = setup_data(dir_prefix + datasets[dataset_idx]['dir_name'],  num_circuits = datasets[dataset_idx]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[dataset_idx]['dir_with_gates'], separate_bits = False, norm = 'no_norm')
dataset = circ[:data_idx], freq[:data_idx]
model = MADE((4,), 20, hidden_size=[160, 160], one_hot_input=True).cuda()
MADE_expt(model, dataset, 0.1, gen_data_2, ep)

In [36]:
import torch.utils.data as data
model = MADE((4,), 2, hidden_size=[16, 16], one_hot_input=True).cuda()
train_loader = data.DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=128)
train_losses, test_losses = train_epochs(model, train_loader, test_loader, 
                                          dict(epochs=20, lr=2e-3))

Epoch 0 Train Loss 0.34731048345565796 Test loss 0.3475
Epoch 1 Train Loss 0.3467375636100769 Test loss 0.3467
Epoch 2 Train Loss 0.34679803252220154 Test loss 0.3466


KeyboardInterrupt: ignored

In [61]:
x = gen_data(ideal_data, 0.001)

In [62]:
x

array([[0., 1., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 1.]])